In [1]:
# IMCOMPLETE: writing report instead

In [16]:
import pandas as pd
import re
from pandasql import sqldf
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve
from sklearn.model_selection import cross_val_score
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [17]:
pd.set_option('display.max_colwidth', 70)
pd.set_option('display.max_rows', None)
pysqldf = lambda q: sqldf(q, globals())

In [18]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/noelsalmeron/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/noelsalmeron/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [19]:
# setting a couple of workbook variables
text_source = "transcript"
tag = "lawsuit"

# text cleaning variables
stop = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [20]:
df_human = pd.read_csv("data/classifications.csv")
df_ml = pd.read_csv("data/transcript_accuracy.csv")

In [21]:
df_dataset = pysqldf(f"""
SELECT dh.*, dml.{text_source}
FROM 
    df_human dh
JOIN
    df_ml dml
ON dh.ID = dml.id
    WHERE description is not NULL
    AND description != 'error code 224003'
""")

In [22]:
len(df_dataset)
df_dataset.head(2)

,ID,description,subject,title,runtime,transcript
0,ffxh0257,Deposition of Jeff Kilper in RE: National Prescription Opiate Liti...,lawsuit; legal activity; litigation; opioids; sales; marketing;,"Deposition of Jeff Kilper, Senior Director for Finance for Special...",1:07:17,"on the record at 12:49 p.m., okay before we took a lunch break mrs..."
1,fghb0039,Camel Cigarettes infomercial on why their cigarettes are superior ...,tobacco; cigarette; marketing;,Camel Cigarettes informercial,00:20:51,it's getting pretty late when I hit the trail when you Snoop aroun...


In [23]:
# lowercase using for loop and new array
#transcript_col = df_dataset['transcript']
#transcripts = []
#for t in transcript_col:
#    transcripts.append(t.lower())
    
# lowercase using apply
df_dataset['transcript'] = df_dataset['transcript'].apply(str.lower, 'transcript')

#df_dataset.head()

In [24]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/noelsalmeron/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [25]:
# using list of stop words from NLTK library 
# + p.m. and a.m. which commonly show up in transcripts
stop_words = stopwords.words('english') + ['we\'re', 'p.m.,', 'a.m.']
#print(stop_words)

def remove_stop_words(str):
    new_str = ''
    for word in str.split(): 
        if word not in stop_words:
            new_str = new_str + word + ' '
    return new_str[:-1] # removes last space

#remove_stop_words('This is a sentence.')

df_dataset['transcript'] = df_dataset['transcript'].apply(remove_stop_words, 'transcript')
df_dataset.head(5)

,ID,description,subject,title,runtime,transcript
0,ffxh0257,Deposition of Jeff Kilper in RE: National Prescription Opiate Liti...,lawsuit; legal activity; litigation; opioids; sales; marketing;,"Deposition of Jeff Kilper, Senior Director for Finance for Special...",1:07:17,record 12:49 okay took lunch break mrs. dilber talking exhibit num...
1,fghb0039,Camel Cigarettes infomercial on why their cigarettes are superior ...,tobacco; cigarette; marketing;,Camel Cigarettes informercial,00:20:51,getting pretty late hit trail snoop around living like ain't 9 5 j...
2,fgxh0257,Deposition of Mark Trudeau in Re: State of Rhode Island v Purdue P...,lawsuit; legal activity; litigation; opioids; sales; marketing;,"Deposition of Mark Trudeau, President and CEO",0:52:40,back record 551 questions witness concludes mr. trudeau compositio...
3,fhfk0146,CNN debate on the ban of smoking on Airplanes. Ban is put in place...,tobacco; cigarette;secondhand smoke; closed environment;,CNN Crossfire debate on smoking in public,0:26:20,tonight washington crossfire saturday smoking skies ifs ands butts...
4,fhgb0191,Compilation of ads for children to say no to smoking.,tobacco; cigarette; youth access; PSA; children; future;,Think. Don't Smoke. ad campaign compilation,0:34:02,hey guys may ask couple questions ever tried cigarettes know reall...


In [26]:
# remove everything that isn't alphanumeric
def alphanumeric(str):
    return re.sub(r'[^a-zA-Z]+', ' ', str)

# alphanumeric("hey 3there00 bud 129.129")

df_dataset['transcript'] = df_dataset['transcript'].apply(alphanumeric, 'transcript')
df_dataset.head(10)

,ID,description,subject,title,runtime,transcript
0,ffxh0257,Deposition of Jeff Kilper in RE: National Prescription Opiate Liti...,lawsuit; legal activity; litigation; opioids; sales; marketing;,"Deposition of Jeff Kilper, Senior Director for Finance for Special...",1:07:17,record okay took lunch break mrs dilber talking exhibit number pag...
1,fghb0039,Camel Cigarettes infomercial on why their cigarettes are superior ...,tobacco; cigarette; marketing;,Camel Cigarettes informercial,00:20:51,getting pretty late hit trail snoop around living like ain t job k...
2,fgxh0257,Deposition of Mark Trudeau in Re: State of Rhode Island v Purdue P...,lawsuit; legal activity; litigation; opioids; sales; marketing;,"Deposition of Mark Trudeau, President and CEO",0:52:40,back record questions witness concludes mr trudeau composition tha...
3,fhfk0146,CNN debate on the ban of smoking on Airplanes. Ban is put in place...,tobacco; cigarette;secondhand smoke; closed environment;,CNN Crossfire debate on smoking in public,0:26:20,tonight washington crossfire saturday smoking skies ifs ands butts...
4,fhgb0191,Compilation of ads for children to say no to smoking.,tobacco; cigarette; youth access; PSA; children; future;,Think. Don't Smoke. ad campaign compilation,0:34:02,hey guys may ask couple questions ever tried cigarettes know reall...
5,fhhb0039,RJ Reynolds tobacco company board discusses advertising Tobacco pr...,tobacco; cigarette; advertise; appeal;,RJ Reynolds Tobacco board meeting on advertisment,0:15:07,hello doug sarah mcclurkin think time program talked know one ciga...
6,fhhh0100,Compilation of ads for Camel cigarettes. Ads were performed by wor...,tobacco; cigarette; break; relief;,Camel cigarette advertisment compilation,0:27:35,ever heard man literally walk wall human fly well looking hundred ...
7,fhhj0223,ABC7 news report on stopping the youth from accessing tobacco prod...,tobacco; cigarette; youth; regulation;,ABC 7 news report: WeCard,0:09:08,top story tonight indiana retail associations joining forces keep ...
8,fhwm0256,Video clip of interview with prescriber discussing their recall of...,opioids; coupon program; efficacy; sales and marketing;,Opiod prescriber interview,0:01:35,tell remember detail happened iraq benefits drug used okay force r...
9,fjhb0039,Standup comedy show with ads inbetween explaining why Winston ciga...,tobacco; cigarette;quality; advertising;,Winston No Bull comedy break,0:17:38,welcome winston bull comedy break guys gonna right watching gerald...


In [27]:
# columns for stem and lemmatizing
# remove one or two character pairs

In [28]:
df_dataset = df_dataset.drop(['description', 'subject'], axis=1)
df_dataset.head(10)

,ID,title,runtime,transcript
0,ffxh0257,"Deposition of Jeff Kilper, Senior Director for Finance for Special...",1:07:17,record okay took lunch break mrs dilber talking exhibit number pag...
1,fghb0039,Camel Cigarettes informercial,00:20:51,getting pretty late hit trail snoop around living like ain t job k...
2,fgxh0257,"Deposition of Mark Trudeau, President and CEO",0:52:40,back record questions witness concludes mr trudeau composition tha...
3,fhfk0146,CNN Crossfire debate on smoking in public,0:26:20,tonight washington crossfire saturday smoking skies ifs ands butts...
4,fhgb0191,Think. Don't Smoke. ad campaign compilation,0:34:02,hey guys may ask couple questions ever tried cigarettes know reall...
5,fhhb0039,RJ Reynolds Tobacco board meeting on advertisment,0:15:07,hello doug sarah mcclurkin think time program talked know one ciga...
6,fhhh0100,Camel cigarette advertisment compilation,0:27:35,ever heard man literally walk wall human fly well looking hundred ...
7,fhhj0223,ABC 7 news report: WeCard,0:09:08,top story tonight indiana retail associations joining forces keep ...
8,fhwm0256,Opiod prescriber interview,0:01:35,tell remember detail happened iraq benefits drug used okay force r...
9,fjhb0039,Winston No Bull comedy break,0:17:38,welcome winston bull comedy break guys gonna right watching gerald...


In [29]:
df_dataset.to_csv('data/cleaned_data.csv', index=False)